In [ ]:
import csv
import pandas as pd
from pandas import Series,DataFrame
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fraud=pd.read_csv('fraud_data.csv')
fraud.columns

In [ ]:
#There are 284807 records, all attributes are numeric, no missing value
fraud.info()

In [ ]:
#The attributes have very different scales
fraud.describe()

In [ ]:
#Obviously, there is a class imbalance problem in the dataset
fraud['Class'].value_counts()

In [ ]:
#Class(mean) differ with different Time
TH=pd.cut(fraud['Time']/3600, np.arange(0,51,5))
grouping = fraud.groupby(TH).mean()
grouping['Class'].plot.bar()
plt.show()

In [ ]:
#Class(mean) differ with different Amount
A=pd.cut(fraud['Amount']/1000, np.arange(0,26))
grouping = fraud.groupby(A).mean()
grouping['Class'].plot.bar()
plt.show()

In [ ]:
X=fraud[fraud.columns[:30]]
y=fraud[fraud.columns[-1:]]

In [ ]:
#As the original dataset is too big that we cannot run it on our computers, we decided to use 10% of the dataset.
##We split the dataset half and half as our training set and test set. 
###train_test_split makse sure that the data is stratified.
from sklearn.model_selection import train_test_split
a, b, c, d = train_test_split(X, y,random_state = 42, test_size = 0.1)
X_train, X_test, y_train, y_test = train_test_split(b, d, random_state = 42, test_size = 0.5)

In [ ]:
#Scale the data with StandardScaler()
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
#Use gridsearchCV to find the best parameter for KNeighborsClassifier
##It takes a long time to run.
knn = KNeighborsClassifier()

grid_values = {'n_neighbors': [2,3,4,5,6,7,8,9,10]}
# default metric to optimize over grid parameters: accuracy
grid_clf_acc = GridSearchCV(knn, param_grid = grid_values)
grid_clf_acc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

In [ ]:
#Show the mean accuracy of validation sets with different n_neighbors
score=grid_clf_acc.grid_scores_
scores=[x[1] for x in score]

n_range=[2,3,4,5,6,7,8,9,10]
plt.figure()
plt.xlabel('n')
plt.ylabel('accuracy')
plt.scatter(n_range, scores, color='orange')
plt.ylim(0.990,1.001)
plt.show()

In [ ]:
#Try different evaluations on the test set based on the best parameter we got.
knnb = KNeighborsClassifier(n_neighbors=5)
knnb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=knnb.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('AUC:',roc_auc_score(y_test,yp))

In [ ]:
#Did the same for Logistic Regression
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10, 20]}
# default metric to optimize over grid parameters: accuracy
grid_clf_acc = GridSearchCV(logit, param_grid = grid_values)
grid_clf_acc.fit(X_train_scaled, y_train.values.reshape(-1))

print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)


In [ ]:
score=grid_clf_acc.grid_scores_
scores=[x[1] for x in score]


C_range=[0.001, 0.01, 0.05, 0.1, 1, 10, 20]
plt.figure()
plt.xlabel('C')
plt.ylabel('accuracy')
plt.scatter(C_range, scores, color='orange')
plt.ylim(0.990,1.001)
plt.show()

In [ ]:
logitb=LogisticRegression(C=0.05)
logitb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=logitb.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('AUC:',roc_auc_score(y_test,yp))

In [ ]:
#The same for SVC linear
from sklearn.svm import SVC
clf = SVC(kernel='linear')
grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

In [ ]:
score=grid_clf_acc.grid_scores_
scores=[x[1] for x in score]

C_range=[0.001, 0.01, 0.05, 0.1, 1, 10]
plt.figure()
plt.xlabel('C')
plt.ylabel('accuracy')
plt.scatter(C_range, scores, color='orange')
plt.ylim(0.991,1.001)
plt.show()

In [ ]:
clfb= SVC(kernel='linear',C=0.01)
clfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=clfb.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('AUC:',roc_auc_score(y_test,yp))

In [ ]:
#The same for SVC rbf
clf = SVC(kernel='rbf')
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

In [ ]:
score=grid_clf_acc.grid_scores_
scores=[x[1] for x in score]

gamma_range=[0.001, 0.01, 0.05, 0.1, 1, 10]
plt.figure()
plt.xlabel('gamma')
plt.ylabel('accuracy')
plt.scatter(gamma_range, scores, color='orange')
plt.ylim(0.990,1.001)
plt.show()

In [ ]:
clfb2= SVC(kernel='rbf',gamma=0.001)
clfb2.fit(X_train_scaled, y_train.values.reshape(-1))
yp=clfb2.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('AUC:',roc_auc_score(y_test,yp))

In [ ]:
#The same for decision tree, but without plotting graph as it will be very difficult
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=42)
grid_values = [
{'max_features':[2, 4, 6, 8], 'max_depth': [10,20,30,40,50,60,70,80,90,100] }]
# default metric to optimize over grid parameters: accuracy
grid_clf_acc = GridSearchCV(tree, param_grid = grid_values)
grid_clf_acc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

In [ ]:
treeb=DecisionTreeClassifier(random_state=42,max_depth=10,max_features=4)
treeb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=treeb.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('AUC:',roc_auc_score(y_test,yp))

In [ ]:
#There is nothing we can change in naive bayes so we caculated the accuracy/recall/AUC directly.
from sklearn.naive_bayes import GaussianNB
nbclf = GaussianNB().fit(X_train_scaled, y_train.values.reshape(-1))
nbclf.score(X_test_scaled, y_test.values.reshape(-1))
yp= nbclf.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('AUC:',roc_auc_score(y_test,yp))

In [ ]:
#Random forests, again we cannot plot the graph.
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
grid_values = [
{'n_estimators': [5, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [10,20,30,40,50,60,70,80,90,100] }]
grid_clf_acc = GridSearchCV(rf, param_grid = grid_values)
grid_clf_acc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

In [ ]:
rfb=RandomForestClassifier(random_state=42,max_depth=10,max_features=8,n_estimators=5)
rfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=rfb.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,yp))
print('recall:',recall_score(y_test,yp))
print('auc:',roc_auc_score(y_test,yp))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#Alternative metric to optimize over grid parameters: AUC
##It takes a long time to run.
###We used gridsearchCV to find the best parameter for KNeighborsClassifier using scoring=AUC
knn = KNeighborsClassifier()

grid_values = {'n_neighbors': [2,3,4,5,6,7,8,9,10]}
# default metric to optimize over grid parameters: accuracy
grid_clf_auc = GridSearchCV(knn, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)


In [ ]:
#Plot the mean AUC for validation sets with different n_neighbors
score=grid_clf_auc.grid_scores_
scores=[x[1] for x in score]

n_range=[2,3,4,5,6,7,8,9,10]
plt.figure()
plt.xlabel('n')
plt.ylabel('AUC')
plt.scatter(n_range, scores, color='orange')
plt.ylim(0.8,1.00)
plt.show()

In [ ]:
#Use the best parameter we got and tried the model on test set
knnb= KNeighborsClassifier(n_neighbors=4)
knnb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=knnb.predict(X_test_scaled)
print('score on test set(AUC):',roc_auc_score(y_test,yp))

In [ ]:
#Did the same thing for logit
logit = LogisticRegression()

grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10, 20]}
# default metric to optimize over grid parameters: accuracy
grid_clf_auc = GridSearchCV(logit, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

In [ ]:
score=grid_clf_auc.grid_scores_
scores=[x[1] for x in score]


C_range=[0.001, 0.01, 0.05, 0.1, 1, 10, 20]
plt.figure()
plt.xlabel('C')
plt.ylabel('AUC')
plt.scatter(C_range, scores, color='orange')
plt.ylim(0.90,1.001)
plt.show()

In [ ]:
logitb= LogisticRegression(C=0.01)
logitb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=logitb.predict(X_test_scaled)
print('score on test set(AUC):',roc_auc_score(y_test,yp))

In [ ]:
#Same for SVC linear
clf = SVC(kernel='linear')
grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10]}
grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

In [ ]:
score=grid_clf_auc.grid_scores_
scores=[x[1] for x in score]

C_range=[0.001, 0.01, 0.05, 0.1, 1, 10]
plt.figure()
plt.xlabel('C')
plt.ylabel('AUC')
plt.scatter(C_range, scores, color='orange')
plt.ylim(0.85,1.001)
plt.show()

In [ ]:
clfb=SVC(kernel='linear', C= 0.1)
clfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=clfb.predict(X_test_scaled)
print('score on test set(AUC):',roc_auc_score(y_test,yp))

In [ ]:
#Same thing for SVC rbf
clf = SVC(kernel='rbf')
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10]}
grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

In [ ]:
score=grid_clf_auc.grid_scores_
scores=[x[1] for x in score]

gamma_range=[0.001, 0.01, 0.05, 0.1, 1, 10]
plt.figure()
plt.xlabel('gamma')
plt.ylabel('AUC')
plt.scatter(gamma_range, scores, color='orange')
plt.ylim(0.7,1.001)
plt.show()

In [ ]:
clfb2=SVC(kernel='rbf', gamma= 0.1)
clfb2.fit(X_train_scaled, y_train.values.reshape(-1))
yp=clfb2.predict(X_test_scaled)
print('score on test set(AUC):',roc_auc_score(y_test,yp))

In [ ]:
#Same on DecisionTreeClassifier, without plotting graph
tree = DecisionTreeClassifier(random_state=42)
grid_values = [
{'max_features':[2, 4, 6, 8], 'max_depth': [10,20,30,40,50,60,70,80,90,100] }]
# default metric to optimize over grid parameters: accuracy
grid_clf_auc = GridSearchCV(tree, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max.AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

In [ ]:
treeb=DecisionTreeClassifier(random_state=42,max_depth=10,max_features=8)
treeb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=treeb.predict(X_test_scaled)
print('score on test set(AUC):',roc_auc_score(y_test,yp))

In [ ]:
#Same for RandomForestClassifier, without plotting graph
rf = RandomForestClassifier(random_state=42)
grid_values = [
{'n_estimators': [5, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [10,20,30,40,50,60,70,80,90,100] }]
grid_clf_auc = GridSearchCV(rf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max.AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

In [ ]:
rfb = RandomForestClassifier(random_state=42, max_depth=20,max_features=2, n_estimators= 20)
rfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=rfb.predict(X_test_scaled)
print('score on test set(AUC):',roc_auc_score(y_test,yp))

In [ ]:
#dummy classifier
from sklearn.dummy import DummyClassifier

# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train_scaled, y_train.values.reshape(-1))
# Therefore the dummy 'most_frequent' classifier always predicts class 0
y_dummy_predictions = dummy_majority.predict(X_test_scaled)
print('score on test set(accuracy):',accuracy_score(y_test,y_dummy_predictions))

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#We prefer AUC rather than recall, so this part is just for our reference
##You may skip this part as it will take extra long time to run
knn = KNeighborsClassifier()

grid_values = {'n_neighbors': [2,3,4,5,6,7,8,9,10]}
# default metric to optimize over grid parameters: accuracy
grid_clf_recall = GridSearchCV(knn, param_grid = grid_values, scoring = 'recall')
grid_clf_recall.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. recall): ', grid_clf_recall.best_params_)
print('Grid best score (recall): ', grid_clf_recall.best_score_)

In [ ]:
knnb= KNeighborsClassifier(n_neighbors=3)
knnb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=knnb.predict(X_test_scaled)
print('score on test set(recall):',recall_score(y_test,yp))

In [ ]:
logit = LogisticRegression()

grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10, 20]}
# default metric to optimize over grid parameters: accuracy
grid_clf_recall = GridSearchCV(logit, param_grid = grid_values, scoring = 'recall')
grid_clf_recall.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. recall): ', grid_clf_recall.best_params_)
print('Grid best score (recall): ', grid_clf_recall.best_score_)

In [ ]:
logitb= LogisticRegression(C=10)
logitb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=logitb.predict(X_test_scaled)
print('score on test set(recall):',recall_score(y_test,yp))

In [ ]:
clf = SVC(kernel='linear')
grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10]}
grid_clf_recall = GridSearchCV(clf, param_grid = grid_values, scoring = 'recall')
grid_clf_recall.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. recall): ', grid_clf_recall.best_params_)
print('Grid best score (recall): ', grid_clf_recall.best_score_)

In [ ]:
clfb=SVC(kernel='linear', C= 0.01)
clfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=clfb.predict(X_test_scaled)
print('score on test set(recall):',recall_score(y_test,yp))

In [ ]:
clf = SVC(kernel='rbf')
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10]}
grid_clf_recall = GridSearchCV(clf, param_grid = grid_values, scoring = 'recall')
grid_clf_recall.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. recall): ', grid_clf_recall.best_params_)
print('Grid best score (recall): ', grid_clf_recall.best_score_)


In [ ]:
clfb=SVC(kernel='rbf', gamma= 0.001)
clfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=clfb.predict(X_test_scaled)
print('score on test set(recall):',recall_score(y_test,yp))

In [ ]:
tree = DecisionTreeClassifier(random_state=42)
grid_values = [
{'max_features':[2, 4, 6, 8], 'max_depth': [10,20,30,40,50,60,70,80,90,100] }]
grid_clf_recall = GridSearchCV(tree, param_grid = grid_values, scoring = 'recall')
grid_clf_recall.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. recall): ', grid_clf_recall.best_params_)
print('Grid best score (recall): ', grid_clf_recall.best_score_)

In [ ]:
treeb=DecisionTreeClassifier(random_state=42,max_depth=10,max_features=8)
treeb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=treeb.predict(X_test_scaled)
print('score on test set(recall):',recall_score(y_test,yp))

In [ ]:
rf = RandomForestClassifier(random_state=42)
grid_values = [
{'n_estimators': [5, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [10,20,30,40,50,60,70,80,90,100] }]
grid_clf_recall = GridSearchCV(rf, param_grid = grid_values, scoring = 'recall')
grid_clf_recall.fit(X_train_scaled, y_train.values.reshape(-1))
print('Grid best parameter (max. recall): ', grid_clf_recall.best_params_)
print('Grid best score (recall): ', grid_clf_recall.best_score_)

In [ ]:
rfb = RandomForestClassifier(random_state=42, max_depth=10,max_features=8, n_estimators= 5)
rfb.fit(X_train_scaled, y_train.values.reshape(-1))
yp=rfb.predict(X_test_scaled)
print('score on test set(recall):',recall_score(y_test,yp))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#We picked some models that performed well on the first part and used ensemble, bagging and pasting to improve the classification accuracy. 
logit_clf=LogisticRegression(C=0.05,random_state=42)
rnd_clf=RandomForestClassifier(max_depth=10, max_features=8, n_estimators=5,random_state=42)
svm_clf=SVC(kernel='rbf',gamma=0.001,probability=True, random_state=42)
knn_clf=KNeighborsClassifier(n_neighbors=5)
tree_clf=DecisionTreeClassifier(max_depth=10, max_features=4, random_state=42)

In [ ]:
#Used hard voting ensemble method
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('logit',logit_clf), ('rf',rnd_clf), ('svc',svm_clf),('knn',knn_clf)], voting='hard')
voting_clf.fit(X_train_scaled, y_train.values.reshape(-1))

In [ ]:
#Accuracy increased a little bit
for clf in (logit_clf, rnd_clf, svm_clf, knn_clf,voting_clf):
    clf.fit(X_train_scaled, y_train.values.reshape(-1))
    yp=clf.predict(X_test_scaled)
    print(clf.__class__.__name__, accuracy_score(y_test.values.reshape(-1), yp))

In [ ]:
#Applied bagging to different models, the accuracies improved a little bit.
from sklearn.ensemble import BaggingClassifier
for clf in (logit_clf, svm_clf, knn_clf):
    bag_clf = BaggingClassifier(clf, bootstrap=True, random_state=42)
    bag_clf.fit(X_train_scaled, y_train.values.reshape(-1))
    yp=bag_clf .predict(X_test_scaled)
    print('With Bagging:',clf.__class__.__name__,'accuracy score:', accuracy_score(y_test.values.reshape(-1), yp))


In [ ]:
#Applied pasting to different models, only the accuracy of SVC increased, others' the same.
for clf in (logit_clf, svm_clf, knn_clf):
    bag_clf = BaggingClassifier(clf, bootstrap=False, random_state=42)
    bag_clf.fit(X_train_scaled, y_train.values.reshape(-1))
    yp=bag_clf .predict(X_test_scaled)
    print('With Pasting:',clf.__class__.__name__,'accuracy score:', accuracy_score(y_test.values.reshape(-1), yp))


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#Tried AdaBoostClassifier and GradientBoosting. We cannot furthur improve the accuracy.
from sklearn.ensemble import AdaBoostClassifier

for clf in (logit_clf, rnd_clf,tree_clf):
    ada_clf = AdaBoostClassifier(clf,learning_rate=0.5, random_state=42)
    ada_clf.fit(X_train_scaled, y_train.values.reshape(-1))
    yp=ada_clf .predict(X_test_scaled)
    print('With Adaboost:',clf.__class__.__name__,'accuracy score:', accuracy_score(y_test.values.reshape(-1), yp))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
Grb_clf = GradientBoostingClassifier(learning_rate=0.5, random_state=42)
Grb_clf.fit(X_train_scaled, y_train.values.reshape(-1))
yp=Grb_clf .predict(X_test_scaled)
print('accuracy score:', accuracy_score(y_test.values.reshape(-1), yp))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#Applied dimension reduction on several models 
from sklearn.decomposition import PCA
pca = PCA(n_components=0.85)
X_train_reduced=pca.fit_transform(X_train_scaled)
X_test_reduced=pca.fit_transform(X_test_scaled)

In [ ]:
#The accuracy increased but AUC and recall dropped dramatically
for clf in (logit_clf, rnd_clf, svm_clf, knn_clf, voting_clf):
    clf.fit(X_train_reduced, y_train.values.reshape(-1))
    yp=clf.predict(X_test_reduced)
    print(clf.__class__.__name__, 'accuracy:', accuracy_score(y_test.values.reshape(-1), yp))
    print(clf.__class__.__name__, 'recall:', recall_score(y_test.values.reshape(-1), yp))
    print(clf.__class__.__name__, 'AUC:', roc_auc_score(y_test.values.reshape(-1), yp))